In [ ]:
!python --version

In [ ]:
!pip install numpy pycocotools opencv-python tqdm tensorboard tensorboardX pyyaml webcolors matplotlib
!pip install torch torchvision --upgrade

In [ ]:
import os
import sys
import shutil
import json
import random
import numpy as np
import cv2
import torch
import yaml
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from PIL import Image, ImageOps, ImageEnhance, UnidentifiedImageError
from sklearn.model_selection import train_test_split

In [ ]:
shutil.rmtree("/kaggle/working")

In [ ]:
# if not os.path.exists('/kaggle/working/images'):
#     os.makedirs('/kaggle/working/images')

# for image_file in os.listdir('/kaggle/input/varroa-mite-detection-on-bees-v3i/images'):
#     shutil.copy(os.path.join('/kaggle/input/varroa-mite-detection-on-bees-v3i/images', image_file), '/kaggle/working/images')

# shutil.copy('/kaggle/input/varroa-mite-detection-on-bees-v3i/_annotations.coco.json', '/kaggle/working/_annotations.coco.json')

In [ ]:
shutil.copytree("/kaggle/input/varroa-yolo/images","/kaggle/working/images")
shutil.copytree("/kaggle/input/varroa-yolo/labels","/kaggle/working/labels")

In [ ]:
classes = ["Varroa"]

with open("/kaggle/working/classes.txt", "w") as f:
    for cls in classes:
        f.write(f"{cls}\n")

print("classes.txt has been created successfully.")

In [ ]:
%cd /kaggle/working
YC_dir = '/kaggle/working/yolo-coco'
if (os.path.exists(YC_dir)):
    shutil.rmtree(YC_dir)
!git clone --depth 1 https://github.com/GharsallahDev/yolo-coco /kaggle/working/yolo-coco

In [ ]:
%cd /kaggle/working/yolo-coco
!python script.py --images /kaggle/working/images --labels /kaggle/working/labels --classes /kaggle/working/classes.txt --output /kaggle/working/coco_annotations.json

In [ ]:
with open("/kaggle/working/coco_annotations.json") as f:
    data = json.load(f)

images = data['images']
annotations = data['annotations']

selected_image = random.choice(images)
image_id = selected_image['id']
file_name = selected_image['file_name']
image_path = os.path.join("/kaggle/working/images", file_name)

print(f"Selected image path: {image_path}")

selected_annotations = [anno for anno in annotations if anno['image_id'] == image_id]

img = Image.open(image_path)
fig, ax = plt.subplots()

ax.imshow(img)

for anno in selected_annotations:
    box = anno['bbox']
    rect = patches.Rectangle((box[0], box[1]), box[2], box[3], linewidth=1, edgecolor='r', facecolor='none')
    ax.add_patch(rect)

plt.show()

In [ ]:
dataset_dir = "/kaggle/working/images"
annotations_path = "/kaggle/working/coco_annotations.json"

with open(annotations_path, 'r') as f:
    coco_data = json.load(f)

bad_images = set()
valid_image_ids = set()
bbox_issues = 0
image_info = {}

for image in coco_data['images']:
    image_path = os.path.join(dataset_dir, image['file_name'])
    image_id = image['id']
    try:
        with Image.open(image_path) as img:
            img.verify()
            image_info[image_id] = {'file_name': image['file_name'], 'width': img.width, 'height': img.height}
            valid_image_ids.add(image_id)
    except Exception:
        bad_images.add(image_id)

for annotation in coco_data['annotations']:
    if annotation['image_id'] not in valid_image_ids:
        continue
    bbox = annotation['bbox']
    img_info = image_info.get(annotation['image_id'], {})
    if not img_info or bbox[0] < 0 or bbox[1] < 0 or (bbox[0] + bbox[2]) > img_info['width'] or (bbox[1] + bbox[3]) > img_info['height']:
        bbox_issues += 1
        bad_images.add(annotation['image_id'])

for image_id in bad_images:
    image_file_name = image_info.get(image_id, {}).get('file_name', None)
    if image_file_name:
        image_path = os.path.join(dataset_dir, image_file_name)
        if os.path.exists(image_path):
            os.remove(image_path)
        valid_image_ids.remove(image_id)

annotated_image_ids = {annotation['image_id'] for annotation in coco_data['annotations']}
unannotated_image_ids = set(image_info.keys()) - annotated_image_ids
for image_id in unannotated_image_ids:
    image_file_name = image_info[image_id]['file_name']
    image_path = os.path.join(dataset_dir, image_file_name)
    if os.path.exists(image_path):
        os.remove(image_path)
    bad_images.add(image_id)

dataset_images = set(os.listdir(dataset_dir))
annotated_image_filenames = {info['file_name'] for image_id, info in image_info.items() if image_id in valid_image_ids}
extra_images_in_dataset = dataset_images - annotated_image_filenames

for image_filename in extra_images_in_dataset:
    os.remove(os.path.join(dataset_dir, image_filename))

coco_data['images'] = [img for img in coco_data['images'] if img['id'] not in bad_images]
coco_data['annotations'] = [anno for anno in coco_data['annotations'] if anno['image_id'] not in bad_images]

with open(annotations_path, 'w') as f:
    json.dump(coco_data, f, indent=4)

print(f"Removed {len(bad_images)} images with issues (corruption, invalid bboxes, or no annotations).")
print(f"Deleted {len(extra_images_in_dataset)} extra images from dataset folder.")
print("Updated annotations file with corrections.")

In [ ]:
!pip install opencv-python
!pip install imgaug>=0.4.0

In [ ]:
%cd /kaggle/working
CR_dir = '/kaggle/working/coco-resize'
if (os.path.exists(CR_dir)):
    shutil.rmtree(CR_dir)
!git clone --depth 1 https://github.com/GharsallahDev/coco-resize /kaggle/working/coco-resize

In [ ]:
%cd /kaggle/working/coco-resize
!python script.py --images_dir="/kaggle/working/images" --annotations_file="/kaggle/working/coco_annotations.json" --image_width=512 --image_height=512 --output_img_dir="/kaggle/working/resized_images" --output_ann_file="/kaggle/working/resized_annotations.coco.json"

In [ ]:
# %cd /kaggle/working
# EDP_dir = '/kaggle/working/EfficientDet'
# if (os.path.exists(EDP_dir)):
#     shutil.rmtree(EDP_dir)
# !git clone --depth 1 https://github.com/GharsallahDev/EfficientDet-Pytorch /kaggle/working/EfficientDet

In [ ]:
EDP_path = '/kaggle/working/EfficientDet'
EDP_url = 'https://github.com/GharsallahDev/EfficientDet-Pytorch'
if not os.path.exists(EDP_path):
    !git clone --depth 1 {EDP_url} {EDP_path}
else:
    %cd {EDP_path}
    !git pull
    %cd /kaggle/working

In [ ]:
images_dir = "/kaggle/working/resized_images"
annotations_path = "/kaggle/working/resized_annotations.coco.json"

def calculate_mean_std(images_dir):
    pixel_sum = np.zeros(3)
    pixel_count = 0

    for filename in os.listdir(images_dir):
        img_path = os.path.join(images_dir, filename)
        cap = cv2.VideoCapture(img_path)

        while True:
            ret, frame = cap.read()
            if not ret:
                break
            frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            pixel_sum += np.sum(frame, axis=(0, 1))
            pixel_count += frame.shape[0] * frame.shape[1]

        cap.release()

    mean = pixel_sum / pixel_count
    var_sum = np.zeros(3)

    for filename in os.listdir(images_dir):
        img_path = os.path.join(images_dir, filename)
        cap = cv2.VideoCapture(img_path)

        while True:
            ret, frame = cap.read()
            if not ret:
                break
            frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            var_sum += np.sum((frame - mean) ** 2, axis=(0, 1))

        cap.release()

    std = np.sqrt(var_sum / pixel_count)
    return mean, std

def analyze_annotations(annotations_path):
    aspect_ratios = []
    object_sizes = []

    with open(annotations_path, 'r') as f:
        data = json.load(f)

    for annotation in data['annotations']:
        bbox_width = annotation['bbox'][2]
        bbox_height = annotation['bbox'][3]
        
        if bbox_width == 0 or bbox_height == 0:
            continue
            
        aspect_ratio = bbox_width / bbox_height
        aspect_ratios.append(aspect_ratio)
        object_sizes.append(bbox_width * bbox_height)

    return aspect_ratios, object_sizes

mean, std = calculate_mean_std(images_dir)

aspect_ratios, object_sizes = analyze_annotations(annotations_path)

anchor_scales = [2 ** 0, 2 ** (1.0 / 3.0), 2 ** (2.0 / 3.0)]
anchor_ratios = [(1.0, 1.0), (1.4, 0.7), (0.7, 1.4)]

print("\nConfiguration:")
print("Mean:", mean)
print("Standard Deviation:", std)
print("Anchor Scales:", anchor_scales)
print("Anchor Ratios:", anchor_ratios)

In [ ]:
mean = [67.18646754, 55.5977657, 45.01203895]
std = [65.85616974, 56.55546839, 50.40794454]
anchor_scales = [1, 1.2599210498948732, 1.5874010519681994]
anchor_ratios = [(1.0, 1.0), (1.4, 0.7), (0.7, 1.4)]

In [ ]:
config_path = "/kaggle/working/EfficientDet/projects/varroa.yml"

yml_content = f"""project_name: varroa
train_set: train
val_set: val
num_gpus: 1

compound_coef: 0

mean: {list(mean)}
std: {list(std)}

anchors_scales: '{anchor_scales}'
anchors_ratios: '{anchor_ratios}'


obj_list: ['Varroa']
"""

with open(config_path, "w") as file:
    file.write(yml_content)

print(f"Configuration file has been saved to {config_path}")

In [ ]:
! cat /kaggle/working/EfficientDet/projects/varroa.yml

In [ ]:
resized_images_dir = "/kaggle/working/resized_images"
annotations_path = "/kaggle/working/resized_annotations.coco.json"
project_dir = "/kaggle/working/EfficientDet/datasets/varroa"

train_dir = os.path.join(project_dir, "train")
val_dir = os.path.join(project_dir, "val")
test_dir = os.path.join(project_dir, "test")

os.makedirs(train_dir, exist_ok=True)
os.makedirs(val_dir, exist_ok=True)
os.makedirs(test_dir, exist_ok=True)
annotations_dir = os.path.join(project_dir, "annotations")
os.makedirs(annotations_dir, exist_ok=True)

with open(annotations_path) as f:
    coco_data = json.load(f)

trainval_images, test_images = train_test_split(coco_data['images'], test_size=0.1, random_state=42)
train_images, val_images = train_test_split(trainval_images, test_size=0.222, random_state=42)

def filter_annotations(annotations, images):
    image_ids = set(img['id'] for img in images)
    return [anno for anno in annotations if anno['image_id'] in image_ids]

train_annotations = filter_annotations(coco_data['annotations'], train_images)
val_annotations = filter_annotations(coco_data['annotations'], val_images)
test_annotations = filter_annotations(coco_data['annotations'], test_images)

def copy_images(image_list, target_dir):
    for image in image_list:
        source_path = os.path.join(resized_images_dir, image['file_name'])
        target_path = os.path.join(target_dir, image['file_name'])
        shutil.copy(source_path, target_path)

copy_images(train_images, train_dir)
copy_images(val_images, val_dir)
copy_images(test_images, test_dir)

for name, subset, images in zip(['train', 'val', 'test'],
                                 [train_annotations, val_annotations, test_annotations],
                                 [train_images, val_images, test_images]):
    subset_path = os.path.join(annotations_dir, f"instances_{name}.json")
    with open(subset_path, 'w') as f:
        json.dump({'images': images, 'annotations': subset, 'categories': coco_data['categories']}, f)
    print(f"Saved {name} annotations to {subset_path}")

In [ ]:
!watch -n1 nvidia-smi

In [ ]:
pip install pycocotools

In [ ]:
shutil.rmtree("/kaggle/working/EfficientDet/logs")

In [ ]:
%cd /kaggle/working/EfficientDet
! python train.py -c 0 -p varroa --head_only False --lr 1e-3 --batch_size 26 --load_weights weights/efficientdet-d0.pth --num_epochs 50 --save_interval 202 --num_workers 2

In [ ]:
%cd /kaggle/working/EfficientDet

In [ ]:
varroa_logs_dir = "/kaggle/working/EfficientDet/logs/varroa"
weight_files = [f for f in os.listdir(varroa_logs_dir) if 'efficientdet' in f and f.endswith('.pth')]
weight_file = sorted(weight_files, key=lambda x: int(x.split('_')[-2]))[-1]
weight_file_path = os.path.join(varroa_logs_dir, weight_file)

test_images_dir = "/kaggle/working/images"
image_files = [file for file in os.listdir(test_images_dir) if file.endswith('.jpg') or file.endswith('.png') or file.endswith('.jpeg')]
random_image = random.choice(image_files)
random_image_path = os.path.join(test_images_dir, random_image)

!python inference.py -p varroa -w {weight_file_path} -i {random_image_path} -r True -m True
inferred_image_path = os.path.join("/kaggle/working/EfficientDet/test",'inferred_' + random_image)
img = Image.open(inferred_image_path)
fig, ax = plt.subplots()
ax.imshow(img)
plt.show()

In [ ]:
%cd /kaggle/working/EfficientDet
! python train.py -c 0 -p varroa --head_only False --lr 1e-3 --batch_size 26 --load_weights {weight_file_path} --num_epochs 20 --save_interval 202 --num_workers 2

In [ ]:
os.listdir("/kaggle/working/EfficientDet/logs/varroa")

In [ ]:
! python coco_eval.py -c 0 -p varroa -w "logs/varroa/efficientdet-d0_43_8888.pth"

In [ ]:
import zipfile

folder_path = "/kaggle/working/EfficientDet/logs/varroa"
output_zip_path = "/kaggle/working/varroa_logs.zip"

with zipfile.ZipFile(output_zip_path, 'w') as zipf:
    for root, _, files in os.walk(folder_path):
        for file in files:
            file_path = os.path.join(root, file)
            zipf.write(file_path, os.path.relpath(file_path, folder_path))

In [ ]:
%cd /kaggle/working
CR_dir = '/kaggle/working/EfficientDet'
if (os.path.exists(CR_dir)):
    shutil.rmtree(CR_dir)
!git clone --depth 1 https://github.com/signatrix/efficientdet.git /kaggle/working/EfficientDet

In [ ]:
resized_images_dir = "/kaggle/working/resized_images"
annotations_path = "/kaggle/working/resized_annotations.coco.json"
project_dir = "/kaggle/working/EfficientDet/COCO"

train_dir = os.path.join(project_dir, "train")
val_dir = os.path.join(project_dir, "val")
test_dir = os.path.join(project_dir, "test")

os.makedirs(train_dir, exist_ok=True)
os.makedirs(val_dir, exist_ok=True)
os.makedirs(test_dir, exist_ok=True)
annotations_dir = os.path.join(project_dir, "annotations")
os.makedirs(annotations_dir, exist_ok=True)

with open(annotations_path) as f:
    coco_data = json.load(f)

trainval_images, test_images = train_test_split(coco_data['images'], test_size=0.1, random_state=42)
train_images, val_images = train_test_split(trainval_images, test_size=0.222, random_state=42)

def filter_annotations(annotations, images):
    image_ids = set(img['id'] for img in images)
    return [anno for anno in annotations if anno['image_id'] in image_ids]

train_annotations = filter_annotations(coco_data['annotations'], train_images)
val_annotations = filter_annotations(coco_data['annotations'], val_images)
test_annotations = filter_annotations(coco_data['annotations'], test_images)

def copy_images(image_list, target_dir):
    for image in image_list:
        source_path = os.path.join(resized_images_dir, image['file_name'])
        target_path = os.path.join(target_dir, image['file_name'])
        shutil.copy(source_path, target_path)

copy_images(train_images, train_dir)
copy_images(val_images, val_dir)
copy_images(test_images, test_dir)

for name, subset, images in zip(['train', 'val', 'test'],
                                 [train_annotations, val_annotations, test_annotations],
                                 [train_images, val_images, test_images]):
    subset_path = os.path.join(annotations_dir, f"instances_{name}.json")
    with open(subset_path, 'w') as f:
        json.dump({'images': images, 'annotations': subset, 'categories': coco_data['categories']}, f)
    print(f"Saved {name} annotations to {subset_path}")

In [ ]:
shutil.rmtree("/kaggle/working/EfficientDet")